In [ ]:
import requests
import random
import math

#Operator porównania klasy string w Python.



In [ ]:
# W pythonie dwa stringi są porównywane symbol po symbolu od lewej do prawej, przy czym każdy jest zamieniany na odpowiadającą liczbę całkowitą w systemie Unicode
# Małe litery j. łacińskiego mają mniejsze odpowiadające im l. cąłkowite niż wielkie.
print('a' > 'A')

True


In [ ]:
print('aB'>'aC')

False


# Modyfikacje klasy węzła

In [ ]:
class Node(object):
  # W konstruktorze klasy węzeł zmieniam nazwę pola number na key, funkcjonalność pozostanie taka sama.
  #   Dodatkowo każdy węzeł będzie miał pole przechowujące wartość - value
  #   Dodałem wskazówki dotyczące typów w liście argumentów konstruktora, niestety Python nie obsługuje typowania statycznego,
  #   więc zostałem zmuszony do zaimplementowania obsługi błędów.
  def __init__(self, key: str=None, value: str=None, parent: Node=None) -> None:
    if not (isinstance(key, str) and isinstance(value, str)):
      raise TypeError('Drzewo BST skrótowców wymaga typu string dla kluczy oraz wartości')
    # klucze mają być skrótami także będą zapisywane z wielkiej litery, ma to znaczenie w przypadku porównań
    self.key = key.upper()
    # Problem identycznych kluczy postanowiłem rozwiązać tak że każda wartość będzie od razu listą (by uniknąć niespójności)
    self.value = [value]
    self.parent = parent
    self.left = None
    self.right = None

# Modyfikacje klasy drzewa

In [ ]:
# Kod z zajęć zmodyfikowany do obsługi drzewa skrótów
class Tree(object):
    def __init__(self, dictionary=None):
        self.dummy = Node('', '')
        # Dla wygody dodałem możliwość automatycznego wypełnienia drzewa za pomocą argumentu konstruktora
        if isinstance(dictionary, dict) and dictionary!={}:
          for k,v in dictionary.items():
            self.Add(k, v)
    
    # Przeciążyłem kilka magicznych metod dla łatwości korzystania z obiektów tej klasy 
    def __repr__(self):
      h = self.Height()
      return f'Drzewo (BST) skrótów, wysokości {h}'
    def __str__(self):
      self.Draw()
      return ''
    def __len__(self):
      return self.Height()

    def Add(self, key, value, nd=None):
        if (nd==None):
            nd = self.dummy.right
        if (nd==None):
            self.dummy.right = Node(key, value, self.dummy)
            return
        if key==nd.key:
            # Jeśli klucz już istnieje, wartość dodawana jest do listy
            nd.value.append(value)
        if nd.key>key:
            if(nd.left==None):
                nd.left = Node(key, value, nd)
                return
            else:
                self.Add(key, value, nd.left)
                return
        else:
            if(nd.right==None):
                nd.right=Node(key, value, nd)
                return
            else:
                self.Add(key, value, nd.right)
    def Delete(self, _nd):
      if _nd==None:
        return
      elif _nd.left==None and _nd.right==None:
        if _nd==_nd.parent.right: _nd.parent.right=None
        elif _nd==_nd.parent.left:  _nd.parent.left=None
        _nd = None
      elif _nd.left==None:
        if _nd==_nd.parent.right: _nd.parent.right=_nd.right
        elif _nd==_nd.parent.left:  _nd.parent.left=_nd.right
        _nd = None
      elif _nd.right==None:
        if _nd==_nd.parent.right: _nd.parent.right=_nd.left
        elif _nd==_nd.parent.left:  _nd.parent.left=_nd.left
        _nd = None
      else:
        nast = self.Consequent(_nd)
        _nd.key, nast.key = nast.key, _nd.key
        self.Delete(nast)

    def Consequent(self, _nd):
      if _nd.right!=None:
        return self.Minimum(_nd.right)
      while _nd.parent!=None and not _nd.parent.left==_nd:
        _nd = _nd.parent
      return _nd.parent

    def Minimum(self, _nd='nic'):
      if _nd=='nic':
        _nd = self.dummy.right
      while _nd.left!=None:
        _nd = _nd.left
      return _nd

    def Find(self, key_to_search, _nd='nic'):
      if (_nd=='nic'):
        _nd = self.dummy.right
      if (_nd==None):
        return None
      if (_nd.key == key_to_search):
        return _nd
      if (_nd.key > key_to_search):
        return self.Find(key_to_search, _nd.left)
      return self.Find(key_to_search, _nd.right)

    def PrintIO(self, _nd='nic'):
      if _nd=='nic':
        _nd = self.dummy.right
      if _nd==None:
        return
      self.PrintIO(_nd.left)
      print(f'{_nd.key}:{_nd.value}', end=", ")
      self.PrintIO(_nd.right)

    def Sum(self, _nd='nic'):
      if _nd=='nic':
        _nd = self.dummy.right
      if _nd==None:
        return 0
      suma = self.Sum(_nd.left) + _nd.key + self.sum(_nd.right)
      return suma
      
    def Height(self, _nd='nic'):
      if _nd=='nic':
        _nd = self.dummy.right
      if _nd==None:
        return 0
      L = self.Height(_nd.left)
      R = self.Height(_nd.right)
      return max(L, R)+1

    def Rotate(self, B):
      if (B is None or B==self.dummy or B==self.dummy.right):
        return
      A = B.parent
      P = A.parent
      if (A.left==B):
        beta = B.right
        B.parent = P
        if P.right==A:
          P.right=B
        else:
          P.left=B
        B.right = A
        A.parent = B
        A.left = beta
        if beta != None:
          beta.parent = A
      else: # rotacja w lewo
        beta = B.left
        B.parent = P
        if P.left==A:
          P.left=B
        else:
          P.right=B
        B.left = A
        A.parent = B
        A.right = beta
        if beta != None:
          beta.parent = A

    def DSW(self):
      cnt = 0
      walker = self.dummy.right
      while walker!=None:
        if walker.left!=None:
          walker = walker.left
          self.Rotate(walker)
        else:
          #print(walker.number)
          #time.sleep(1)
          walker = walker.right
          cnt+=1
      m = 2**(int(math.log2(cnt+1)))-1

      walker = self.dummy.right
      for _ in range(cnt-m):
        walker = walker.right
        self.Rotate(walker)
        walker=walker.right
      m//=2
      
      while(m>0):
        walker=self.dummy.right
        for _ in range(m):
          walker = walker.right
          self.Rotate(walker)
          walker = walker.right
        m//=2

    def Draw(self, _nd='nic', depth=0):
      if _nd=='nic':
        _nd = self.dummy.right
      if _nd==None:
        return
      self.Draw(_nd.right, depth+1)
      for i in range(depth):
        print('   ', end='')
      print(_nd.key)
      self.Draw(_nd.left, depth+1)

    def DrawValues(self, _nd='nic', depth=0):
      if _nd=='nic':
        _nd = self.dummy.right
      if _nd==None:
        return
      self.DrawValues(_nd.right, depth+1)
      for i in range(depth):
        print('    ', end='')
      print(_nd.value)
      self.DrawValues(_nd.left, depth+1)
    ##############################
    def CountLeaves(self, _nd='nic'):
      if _nd=='nic':
        _nd = self.dummy.right
      if _nd==None:
        return 0
      if _nd.left==None and _nd.right==None:
        return 1
      cnt = self.CountLeaves(_nd.left) + self.CountLeaves(_nd.right)
      return cnt

# Generowanie nazw oraz skrótów

In [ ]:
word_site = "https://www.mit.edu/~ecprice/wordlist.10000"
response = requests.get(word_site)
words = response.text.splitlines()

In [ ]:
# Poniższy kod losuje nazwy o losowej długości z wyrazów wybieranych losowo, które trafiają do słownika. Z nazw tworzne są skróty, które stają się kluczami słownika.
def generuj_nazwy(ile_nazw=10, max_wyrazow=5):
  skrot_nazwa = {}
  for _ in range(ile_nazw):
    nazwa = ''
    skrot = ''
    for _ in range(random.randint(2,max_wyrazow)):
      w = random.choice(words).capitalize()
      nazwa += w+' '
      skrot += w[0].upper()
    skrot_nazwa[skrot] = nazwa[:-1]
  return skrot_nazwa

In [ ]:
nazwy = generuj_nazwy(11, 6)
nazwy

{'BTIPS': 'Booking Tide Implement Position Sk',
 'EETVBK': 'Emission Exec Traditions Vol Bottom Korean',
 'MN': 'Mission Normal',
 'ALDRC': 'Avoid Liked Do Remedies Cyprus',
 'VARSFH': 'Virus Accounts Romantic Sparc Fx Harbor',
 'FDVO': 'Feeds Does Vernon Own',
 'EMCTD': 'Expires Miniature Collectibles Thailand Doctrine',
 'SIPT': 'Save Immune Passion Test',
 'HDICAE': 'Highlight Directors Ima Crap Affiliation Experiment',
 'CGFP': 'Congressional Gradually Fighting Plaintiff',
 'CDARQE': 'Collecting Danger Adoption Ruled Qui Except'}

# Testowanie drzewa

In [ ]:
drzewo = Tree(nazwy)

In [ ]:
len(drzewo)

5

In [ ]:
print(drzewo)

         VARSFH
            SIPT
      MN
            HDICAE
         FDVO
            EMCTD
   EETVBK
      CGFP
         CDARQE
BTIPS
   ALDRC



In [ ]:
drzewo.PrintIO()

ALDRC:['Avoid Liked Do Remedies Cyprus'], BTIPS:['Booking Tide Implement Position Sk'], CDARQE:['Collecting Danger Adoption Ruled Qui Except'], CGFP:['Congressional Gradually Fighting Plaintiff'], EETVBK:['Emission Exec Traditions Vol Bottom Korean'], EMCTD:['Expires Miniature Collectibles Thailand Doctrine'], FDVO:['Feeds Does Vernon Own'], HDICAE:['Highlight Directors Ima Crap Affiliation Experiment'], MN:['Mission Normal'], SIPT:['Save Immune Passion Test'], VARSFH:['Virus Accounts Romantic Sparc Fx Harbor'], 

In [ ]:
drzewo.Add('CDARQF','Cc Dd Aa Rr Qq Ff')

In [ ]:
print(drzewo)

         VARSFH
            SIPT
      MN
            HDICAE
         FDVO
            EMCTD
   EETVBK
      CGFP
            CFU
               CDARQF
         CDARQE
BTIPS
   ALDRC



Jak widać dodawanie działa prawidłowo element 'CDARQF' stał się prawą gałęzią 'CDARQE', gdyż 'F'>'E'

In [ ]:
element_do_usuniecia = drzewo.Find('FDVO')

In [ ]:
drzewo.Delete(element_do_usuniecia)

In [ ]:
print(drzewo)

         VARSFH
            SIPT
      MN
         HDICAE
            EMCTD
   EETVBK
      CGFP
            CFU
               CDARQF
         CDARQE
BTIPS
   ALDRC



Element został usnięty prawidłowo.

Przetestuje teraz metodę DSW do równoważenia drzewa.

In [ ]:
drzewo = Tree(generuj_nazwy(1_000, 6))
drzewo

Drzewo (BST) skrótów, wysokości 23

In [ ]:
drzewo.DSW()
drzewo

Drzewo (BST) skrótów, wysokości 10

In [ ]:
math.log(1_000, 2)

9.965784284662087

Jak widać wysokość drzewa uległa dużej poprawie, jest bliska log2(n).

In [ ]:
!jupyter nbconvert --to html /content/Projekt_nr2.ipynb

[NbConvertApp] Converting notebook /content/Projekt_nr2.ipynb to html
[NbConvertApp] Writing 639034 bytes to /content/Projekt_nr2.html
